In [1]:
"""
The MIT License (MIT)
Copyright (c) 2021 NVIDIA
Permission is hereby granted, free of charge, to any person obtaining a copy of
this software and associated documentation files (the "Software"), to deal in
the Software without restriction, including without limitation the rights to
use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of
the Software, and to permit persons to whom the Software is furnished to do so,
subject to the following conditions:
The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS
FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR
COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER
IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN
CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.
"""


'\nThe MIT License (MIT)\nCopyright (c) 2021 NVIDIA\nPermission is hereby granted, free of charge, to any person obtaining a copy of\nthis software and associated documentation files (the "Software"), to deal in\nthe Software without restriction, including without limitation the rights to\nuse, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of\nthe Software, and to permit persons to whom the Software is furnished to do so,\nsubject to the following conditions:\nThe above copyright notice and this permission notice shall be included in all\ncopies or substantial portions of the Software.\nTHE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR\nIMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS\nFOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR\nCOPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER\nIN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OU

This code example is very similar to c6e1_boston but the network is modified to use more layers with more neurons as well as dropout regularization. More context for this code example can be found in the section "Experiment: Deeper and Regularized Models for House Price Prediction" in Chapter 6 in the book Learning Deep Learning by Magnus Ekman (ISBN: 9780137470358).

One thing to note is that Dropout layers do behave differently between training and test. Therefore it is crucial that the training loop calls model.train() and model.eval() correctly. However, given that we started doing that in our very first PyTorch example, our training loop already does this and no changes are needed.


In [ ]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
from utilities import train_model

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
EPOCHS = 500
BATCH_SIZE = 16

# Read and standardize the data.
boston_housing = load_boston()
data = boston_housing.get('data')
target = boston_housing.get('target')

raw_x_train, raw_x_test, y_train, y_test = train_test_split(
    data, target, test_size=0.2, random_state=0)

# Convert to same precision as model.
raw_x_train = raw_x_train.astype(np.float32)
raw_x_test = raw_x_test.astype(np.float32)
y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)
y_train = np.reshape(y_train, (-1, 1))
y_test = np.reshape(y_test, (-1, 1))

x_mean = np.mean(raw_x_train, axis=0)
x_stddev = np.std(raw_x_train, axis=0)
x_train = (raw_x_train - x_mean) / x_stddev
x_test = (raw_x_test - x_mean) / x_stddev

# Create Dataset objects.
trainset = TensorDataset(torch.from_numpy(x_train),
                         torch.from_numpy(y_train))
testset = TensorDataset(torch.from_numpy(x_test),
                        torch.from_numpy(y_test))

# Create model.
model = nn.Sequential(
    nn.Linear(13, 128),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(128, 128),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(128, 64),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(64, 1)
)

# Initialize weights.
for module in model.modules():
    if isinstance(module, nn.Linear):
        nn.init.xavier_uniform_(module.weight)
        nn.init.constant_(module.bias, 0.0)

# Loss function and optimizer
optimizer = torch.optim.Adam(model.parameters())
loss_function = nn.MSELoss()

# Train model.
train_model(model, device, EPOCHS, BATCH_SIZE, trainset, testset,
            optimizer, loss_function, 'mae')

# Print first 4 predictions.
inputs = torch.from_numpy(x_test)
inputs = inputs.to(device)
outputs = model(inputs)
for i in range(0, 4):
    print('Prediction: %4.2f' % outputs.data[i].item(),
         ', true value: %4.2f' % y_test[i].item())


/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np

        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.e.

Epoch 1/500 loss: 528.7918 - mae: 20.6121 - val_loss: 391.7839 - val_mae: 16.3903
Epoch 2/500 loss: 235.8860 - mae: 12.0762 - val_loss: 102.0132 - val_mae: 6.6844
Epoch 3/500 loss: 75.3459 - mae: 6.4953 - val_loss: 56.7289 - val_mae: 5.2625
Epoch 4/500 loss: 48.0323 - mae: 5.3019 - val_loss: 44.1069 - val_mae: 4.4903
Epoch 5/500 loss: 47.6120 - mae: 4.8705 - val_loss: 43.0301 - val_mae: 4.2726
Epoch 6/500 loss: 39.7270 - mae: 4.7117 - val_loss: 35.9447 - val_mae: 3.8543
Epoch 7/500 loss: 37.6954 - mae: 4.3591 - val_loss: 34.5589 - val_mae: 3.7502
Epoch 8/500 loss: 34.5311 - mae: 4.2647 - val_loss: 30.4697 - val_mae: 3.4902
Epoch 9/500 loss: 31.3899 - mae: 4.1682 - val_loss: 29.3391 - val_mae: 3.3621
Epoch 10/500 loss: 25.8267 - mae: 3.8595 - val_loss: 28.4004 - val_mae: 3.3066
Epoch 11/500 loss: 31.3393 - mae: 4.1442 - val_loss: 32.0590 - val_mae: 3.4985
Epoch 12/500 loss: 32.4804 - mae: 4.2546 - val_loss: 27.0877 - val_mae: 3.1864
Epoch 13/500 loss: 33.4283 - mae: 4.2237 - val_loss: 3

Epoch 106/500 loss: 21.6816 - mae: 3.3529 - val_loss: 18.6497 - val_mae: 2.7201
Epoch 107/500 loss: 17.6176 - mae: 3.1405 - val_loss: 17.8434 - val_mae: 2.6651
Epoch 108/500 loss: 17.6649 - mae: 3.0312 - val_loss: 19.0944 - val_mae: 2.7086
Epoch 109/500 loss: 20.8016 - mae: 3.2888 - val_loss: 18.6484 - val_mae: 2.6847
Epoch 110/500 loss: 24.5979 - mae: 3.6443 - val_loss: 19.4423 - val_mae: 2.7400
Epoch 111/500 loss: 19.0437 - mae: 3.0521 - val_loss: 17.8507 - val_mae: 2.5451
Epoch 112/500 loss: 20.7821 - mae: 3.3014 - val_loss: 19.7112 - val_mae: 2.6750
Epoch 113/500 loss: 19.6519 - mae: 3.2376 - val_loss: 22.2278 - val_mae: 2.8852
Epoch 114/500 loss: 20.5635 - mae: 3.2511 - val_loss: 18.4514 - val_mae: 2.5386
Epoch 115/500 loss: 21.1993 - mae: 3.3937 - val_loss: 16.5725 - val_mae: 2.5098
Epoch 116/500 loss: 18.8337 - mae: 3.1868 - val_loss: 19.8208 - val_mae: 2.7913
Epoch 117/500 loss: 18.7367 - mae: 3.1133 - val_loss: 21.7037 - val_mae: 2.9443
Epoch 118/500 loss: 17.3623 - mae: 3.150

Epoch 213/500 loss: 17.6107 - mae: 3.0310 - val_loss: 14.8028 - val_mae: 2.3787
Epoch 214/500 loss: 15.8779 - mae: 2.8889 - val_loss: 15.3416 - val_mae: 2.3946
Epoch 215/500 loss: 13.4280 - mae: 2.7265 - val_loss: 15.3825 - val_mae: 2.4332
Epoch 216/500 loss: 17.9087 - mae: 3.0671 - val_loss: 16.5514 - val_mae: 2.4898
Epoch 217/500 loss: 16.0003 - mae: 2.8206 - val_loss: 15.3008 - val_mae: 2.4369
Epoch 218/500 loss: 14.8004 - mae: 2.8549 - val_loss: 19.2906 - val_mae: 2.5786
Epoch 219/500 loss: 17.8437 - mae: 3.1800 - val_loss: 15.7073 - val_mae: 2.5405
Epoch 220/500 loss: 17.3707 - mae: 3.0806 - val_loss: 16.6512 - val_mae: 2.5616
Epoch 221/500 loss: 16.2341 - mae: 2.8955 - val_loss: 14.8031 - val_mae: 2.4326
Epoch 222/500 loss: 16.3324 - mae: 2.9020 - val_loss: 18.6042 - val_mae: 2.4935
Epoch 223/500 loss: 13.3050 - mae: 2.6097 - val_loss: 17.0207 - val_mae: 2.4058
Epoch 224/500 loss: 18.4585 - mae: 3.1553 - val_loss: 15.6648 - val_mae: 2.4930
Epoch 225/500 loss: 13.6843 - mae: 2.761

Epoch 319/500 loss: 14.1668 - mae: 2.7250 - val_loss: 15.5331 - val_mae: 2.3212
Epoch 320/500 loss: 14.0250 - mae: 2.7040 - val_loss: 14.6491 - val_mae: 2.3049
Epoch 321/500 loss: 13.5932 - mae: 2.6268 - val_loss: 15.7080 - val_mae: 2.3648
Epoch 322/500 loss: 12.2745 - mae: 2.6054 - val_loss: 15.0417 - val_mae: 2.2942
Epoch 323/500 loss: 13.8232 - mae: 2.7473 - val_loss: 14.7171 - val_mae: 2.3133
Epoch 324/500 loss: 12.4761 - mae: 2.6364 - val_loss: 14.6148 - val_mae: 2.3349
Epoch 325/500 loss: 14.7562 - mae: 2.8201 - val_loss: 18.6117 - val_mae: 2.5094
Epoch 326/500 loss: 14.0376 - mae: 2.7366 - val_loss: 17.4057 - val_mae: 2.5441
Epoch 327/500 loss: 15.4164 - mae: 2.8805 - val_loss: 15.3461 - val_mae: 2.3821
Epoch 328/500 loss: 16.3614 - mae: 2.8958 - val_loss: 14.3238 - val_mae: 2.3527
Epoch 329/500 loss: 14.5800 - mae: 2.7166 - val_loss: 14.4388 - val_mae: 2.2940
Epoch 330/500 loss: 13.4236 - mae: 2.7281 - val_loss: 14.1279 - val_mae: 2.2849
Epoch 331/500 loss: 10.9349 - mae: 2.503

Epoch 426/500 loss: 13.3921 - mae: 2.4957 - val_loss: 15.6287 - val_mae: 2.2833
Epoch 427/500 loss: 11.7886 - mae: 2.4857 - val_loss: 16.5754 - val_mae: 2.4767
Epoch 428/500 loss: 15.0473 - mae: 2.6877 - val_loss: 16.7790 - val_mae: 2.4827
Epoch 429/500 loss: 12.6346 - mae: 2.5941 - val_loss: 15.4071 - val_mae: 2.4571
Epoch 430/500 loss: 12.4160 - mae: 2.4844 - val_loss: 14.6055 - val_mae: 2.4221
Epoch 431/500 loss: 11.0467 - mae: 2.3962 - val_loss: 18.7185 - val_mae: 2.5016
Epoch 432/500 loss: 12.2605 - mae: 2.4304 - val_loss: 16.7311 - val_mae: 2.4126
Epoch 433/500 loss: 11.5512 - mae: 2.5328 - val_loss: 15.4796 - val_mae: 2.3573
Epoch 434/500 loss: 11.5430 - mae: 2.4744 - val_loss: 16.4764 - val_mae: 2.3705
Epoch 435/500 loss: 13.4214 - mae: 2.4469 - val_loss: 17.7044 - val_mae: 2.5071
Epoch 436/500 loss: 11.2812 - mae: 2.4194 - val_loss: 15.6239 - val_mae: 2.4566
Epoch 437/500 loss: 16.2343 - mae: 2.8790 - val_loss: 19.6626 - val_mae: 2.5919
Epoch 438/500 loss: 13.5472 - mae: 2.719